In [389]:
import pandas as pd
import numpy as np
import re

In [390]:
# pd.set_option('display.max_colwidth', None)

In [391]:
pd.reset_option('display.max_colwidth')

In [392]:
master_data = {
    "gratka_2023_master" : "../original_data_2023/gratka/ALL_DATA/gdansk_gratka_master_data.json",
}

for key, value in master_data.items():
    master_data[key] = pd.read_json(value)  

gratka_df = master_data["gratka_2023_master"]

In [393]:
df = gratka_df

In [394]:
df.shape

(1647, 31)

In [395]:
df.head()

,link,rent,description,address,title,additional_info,Lokalizacja,Powierzchnia w m2,Liczba pokoi,Piętro,...,Materiał budynku,Liczba miejsc parkingowych,Czy mieszkanie ma łazienkę?,Możliwy kontakt w języku,"Opłaty (czynsz administracyjny, media)",Stan instalacji,Stan łazienki,Okna,Głośność,Usytuowanie względem stron świata
0,https://gratka.pl/nieruchomosci/mieszkanie-gda...,6 500,"Inwestycja BRABANK zachwyca lokalizacją, archi...",Gdańsk pomorskie,Mieszkanie Gdańsk ul. Stara Stocznia,[],"Gdańsk, ...","68,69 m2",3.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://gratka.pl/nieruchomosci/mieszkanie-gda...,2 450,"homfi ma przyjemność zaprezentować jasne, prze...",Gdańsk Brzeźno pomorskie,Mieszkanie Gdańsk Brzeźno ul. Dunikowskiego,[],"Gdańsk, ...",47 m2,2.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://gratka.pl/nieruchomosci/mieszkanie-gda...,6 800,Zamieszkaj w stylowej kamienicy. Zapraszamy do...,Gdańsk Stare Miasto pomorskie,Mieszkanie Gdańsk Stare Miasto ul. Tobiasza,[],"Gdańsk, ...","100,40 m2",4.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://gratka.pl/nieruchomosci/mieszkanie-gda...,5 400,Lokal 125 m/2 w Gdańsk Oliwa u zbiegu ul Al. G...,Gdańsk Oliwa pomorskie,Mieszkanie Gdańsk Oliwa ul. Grunwaldzka,[],"Gdańsk, ...",125 m2,4.0,2,...,cegła,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://gratka.pl/nieruchomosci/mieszkanie-gda...,2 500,--==Rezerwacja==-- --==Rezerwacja==-- --==Rese...,Gdańsk Piecki-Migowo pomorskie,Mieszkanie Gdańsk Piecki-Migowo ul. Myśliwska,[],"Gdańsk, ...",40 m2,2.0,3,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [396]:
gratka_title_to_location_lowercase = {
    "piecki-migowo ul. bulońska": "Piecki Migowo - Suchanino - Morena",
    "piecki-migowo ul. piecewska": "Piecki Migowo - Suchanino - Morena",
    "piecki-migowo ul. wileńska": "Piecki Migowo - Suchanino - Morena",
    "piecki-migowo ul. stanisława lema": "Piecki Migowo - Suchanino - Morena",
    
    "ul. jana kilińskiego": "Wrzeszcz",
    "ul. panieńska": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "ul. piecewska": "Piecki Migowo - Suchanino - Morena",
    "ul. stara stocznia": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "ul. myśliwska": "Piecki Migowo - Suchanino - Morena",
    "ul. pastelowa": "Chelm - Lostowice - Poludnie",
    "ul. toruńska": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "ul. szeroka": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "ul. ks. jerzego popiełuszki 26": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "ul. mariana kołodzieja": "Chelm - Lostowice - Poludnie",
    "ul. tobiasza": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "ul. starowiejska": "Letnica",
    "ul. sadowa": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "ul. szczecińska": "Przymorze",
    "ul. goska": "Jasien - Matarnia - Kokoszki - Osowa",            
    "ul. tandeta": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "ul. sienna grobla": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "ul. nadmorski dwór": "Brzezno",
    "ul. konrada guderskiego": "Jasien - Matarnia - Kokoszki - Osowa",
    "ul. popiełuszki": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "ul. chmielna": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "al. rzeczypospolitej": "Zaspa",
    "ul. płk. wilka-krzyżanowskiego": "Oliwa - Strzyza",
    "ul. bolesława chrobrego": "Wrzeszcz",
    "ul. turystyczna 60": "Orunia",
    "ul. św. barbary": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "ul. kołodzieja 56": "Chelm - Lostowice - Poludnie",
    "ul. wejhera": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "ul. kilińskiego 9": "Wrzeszcz"
}

gratka_gda_districts_mapping = {
    "Śródmieście": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "StareMiasto": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "GłówneMiasto": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "DolneMiasto": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "StarePrzedmieście": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "DługieOgrody": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "WyspaSpichrzów": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    
    "Przymorze": "Przymorze",
    "PrzymorzeWielkie": "Przymorze",
    "PrzymorzeMałe": "Przymorze",
    
    "Łostowice": "Chelm - Lostowice - Poludnie",
    "Ujeścisko": "Chelm - Lostowice - Poludnie",
    "Chełm": "Chelm - Lostowice - Poludnie",
    "Zakoniczyn": "Chelm - Lostowice - Poludnie",
    "Kowale": "Chelm - Lostowice - Poludnie",
    
    "Wrzeszcz": "Wrzeszcz",
    "WrzeszczDolny": "Wrzeszcz",
    "WrzeszczGórny": "Wrzeszcz",
    
    "Brzeźno": "Brzezno",
    
    "Jelitkowo": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Żabianka": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    
    "Siedlce": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Aniołki": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "WzgórzeMickiewicza": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    
    "Jasień": "Jasien - Matarnia - Kokoszki - Osowa",
    "Osowa": "Jasien - Matarnia - Kokoszki - Osowa",
    "Kokoszki": "Jasien - Matarnia - Kokoszki - Osowa",
    "Kiełpinek": "Jasien - Matarnia - Kokoszki - Osowa",
    "Matarnia": "Jasien - Matarnia - Kokoszki - Osowa",
    
    "Zaspa-Rozstaje": "Zaspa",
    "Zaspa": "Zaspa",
    
    "Suchanino": "Piecki Migowo - Suchanino - Morena",
    "Brętowo": "Piecki Migowo - Suchanino - Morena",
    "Piecki": "Piecki Migowo - Suchanino - Morena",
    "Migowo": "Piecki Migowo - Suchanino - Morena",
    "Niedźwiednik": "Piecki Migowo - Suchanino - Morena",
    
    "Letnica": "Letnica",
    
    "Oliwa": "Oliwa - Strzyza",
    "Strzyża": "Oliwa - Strzyza",
    
    "Orunia": "Orunia",
    "Maćkowy": "Orunia",
    "GórkiZachodnie": "Orunia",
    "Olszynka": "Orunia",
    
    "NowyPort": "Przerobka - Stogi - Nowy Port",
    "Przeróbka": "Przerobka - Stogi - Nowy Port",
    "Stogi": "Przerobka - Stogi - Nowy Port",
    
    np.nan: "unknown"
}

gratka_rooms_mapping = {
    3: 3,
    2: 2,
    1: 1,
    4: 4,
    5: 4,
    6: 4,
    7: 4
}

gratka_private_seller_mapping = {
    1.0: True,
    np.nan: False
}

gratka_furnished_mapping = {
    "Tak": True,
}


gratka_level_mapping = {
    "parter": 0,
    "1": 1, "2": 2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9, "10":10,
    "11": 11,
    "12": 11,
    "13": 11,
    "14": 11,
    "15": 11,
    "16": 11,
    "17": 11,
    "poddasze": 11,
}

gratka_building_type_mapping = {
    "blok": "unit",
    "apartamentowiec": "apartment_building",
    "kamienica": "tenement",
    "domwielorodzinny/szeregowiec": "house",
    "loft": "unit",
    np.nan: "unknown"
}

In [397]:
dirty_data = df.copy()

In [398]:
dirty_data["Typ zabudowy"].value_counts()

Typ zabudowy
apartamentowiec                  535
blok                             433
kamienica                        127
dom wielorodzinny/szeregowiec      8
loft                               1
Name: count, dtype: int64

In [399]:
dirty_data["Piętro"].value_counts()

Piętro
1           360
2           342
3           222
parter      220
4           176
5           103
6            49
7            40
9            17
11           14
8            14
10           14
12           14
16           10
17           10
13            7
15            6
14            5
poddasze      1
Name: count, dtype: int64

In [400]:
dirty_data.isna().sum()

link                                         0
rent                                         0
description                                  0
address                                      0
title                                        0
additional_info                              0
Lokalizacja                                  0
Powierzchnia w m2                            1
Liczba pokoi                                 4
Piętro                                      23
Rok budowy                                 433
Typ zabudowy                               543
Liczba pięter w budynku                     84
Dostępność od                              994
Numer referencyjny                          95
Udogodnienia                               441
Zaktualizowane                               0
Dodane                                       0
Stan                                      1341
Miejsce parkingowe                        1366
Forma kuchni                              1254
Materiał budy

In [401]:
dirty_data.columns

Index(['link', 'rent', 'description', 'address', 'title', 'additional_info',
       'Lokalizacja', 'Powierzchnia w m2', 'Liczba pokoi', 'Piętro',
       'Rok budowy', 'Typ zabudowy', 'Liczba pięter w budynku',
       'Dostępność od', 'Numer referencyjny', 'Udogodnienia', 'Zaktualizowane',
       'Dodane', 'Stan', 'Miejsce parkingowe', 'Forma kuchni',
       'Materiał budynku', 'Liczba miejsc parkingowych',
       'Czy mieszkanie ma łazienkę?', 'Możliwy kontakt w języku',
       'Opłaty (czynsz administracyjny, media)', 'Stan instalacji',
       'Stan łazienki', 'Okna', 'Głośność',
       'Usytuowanie względem stron świata'],
      dtype='object')

In [402]:
dirty_data.head()

,link,rent,description,address,title,additional_info,Lokalizacja,Powierzchnia w m2,Liczba pokoi,Piętro,...,Materiał budynku,Liczba miejsc parkingowych,Czy mieszkanie ma łazienkę?,Możliwy kontakt w języku,"Opłaty (czynsz administracyjny, media)",Stan instalacji,Stan łazienki,Okna,Głośność,Usytuowanie względem stron świata
0,https://gratka.pl/nieruchomosci/mieszkanie-gda...,6 500,"Inwestycja BRABANK zachwyca lokalizacją, archi...",Gdańsk pomorskie,Mieszkanie Gdańsk ul. Stara Stocznia,[],"Gdańsk, ...","68,69 m2",3.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://gratka.pl/nieruchomosci/mieszkanie-gda...,2 450,"homfi ma przyjemność zaprezentować jasne, prze...",Gdańsk Brzeźno pomorskie,Mieszkanie Gdańsk Brzeźno ul. Dunikowskiego,[],"Gdańsk, ...",47 m2,2.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://gratka.pl/nieruchomosci/mieszkanie-gda...,6 800,Zamieszkaj w stylowej kamienicy. Zapraszamy do...,Gdańsk Stare Miasto pomorskie,Mieszkanie Gdańsk Stare Miasto ul. Tobiasza,[],"Gdańsk, ...","100,40 m2",4.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://gratka.pl/nieruchomosci/mieszkanie-gda...,5 400,Lokal 125 m/2 w Gdańsk Oliwa u zbiegu ul Al. G...,Gdańsk Oliwa pomorskie,Mieszkanie Gdańsk Oliwa ul. Grunwaldzka,[],"Gdańsk, ...",125 m2,4.0,2,...,cegła,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://gratka.pl/nieruchomosci/mieszkanie-gda...,2 500,--==Rezerwacja==-- --==Rezerwacja==-- --==Rese...,Gdańsk Piecki-Migowo pomorskie,Mieszkanie Gdańsk Piecki-Migowo ul. Myśliwska,[],"Gdańsk, ...",40 m2,2.0,3,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [422]:
def convert_numeric_to_float(convertable_string):
    return float(convertable_string.replace("zł", "").replace(" ", "").replace("\ndonegocjacji", ""))

def clear_gratka(df: pd.DataFrame):
    df.dropna(subset=['Liczba pokoi', 'Powierzchnia w m2', 'Piętro'], inplace=True)
    
    df['Piętro'] = df['Piętro'].fillna(1001)
    df["Typ zabudowy"] = df["Typ zabudowy"].fillna("Unknown")
    
    df.drop(
        labels=[
            'Usytuowanie względem stron świata',
            'Stan',
            'Miejsce parkingowe',
            'Forma kuchni',
            'Materiał budynku',
            'Liczba miejsc parkingowych',
            'Czy mieszkanie ma łazienkę?',
            'Możliwy kontakt w języku',
            'Opłaty (czynsz administracyjny, media)',
            'Stan instalacji',
            'Stan łazienki',
            'Okna',
            'Dostępność od',
            'Zaktualizowane',
            'Głośność',
            'link',
            'description',
            'Numer referencyjny',
            'Dodane',
            'Udogodnienia',
            'Rok budowy',
            'additional_info',
            'Liczba pięter w budynku',
            'Lokalizacja'
        ],
        inplace= True,
        axis= 1
    )

    # building_type
    df = df.rename(mapper={"Typ zabudowy": "building_type"}, axis=1)
    df["building_type"] = df["building_type"].str.replace(" ", "").str.lower()
    df["building_type"] = df["building_type"].replace(gratka_building_type_mapping)
        
    # surface
    df = df.rename(mapper={"Powierzchnia w m2": "surface"}, axis=1)
    df['surface'] = df['surface'].str.replace("m²", "").str.replace("m2", "").str.replace(" ", "").str.replace(",", ".").astype(float)

    # level 
    df = df.rename(mapper={"Piętro": "level"}, axis=1)
    df['level'] = df['level'].map(gratka_level_mapping)
    
    # rooms 
    df = df.rename(mapper={"Liczba pokoi": "rooms"}, axis=1)
    df['rooms'] = df['rooms'].map(gratka_rooms_mapping)
    df['rooms'] = df['rooms'].astype(int)
    
    # rent
    df['rent'] = df['rent'].apply(convert_numeric_to_float)

#     # rent_extra
#     df = df.rename(mapper={"Czynsz (dodatkowo)": "rent_extra"}, axis=1)
#     df['rent_extra'] = df['rent_extra'].apply(convert_numeric_to_float)

    # location
    df = df.rename(mapper={"address": "location"}, axis=1)
    df['location'] = df['location'].str.lstrip('Gdańsk').str.rstrip('pomorskie').str.replace(' ', '').replace(',', '')
    df['location'] = df['location'].map(gratka_gda_districts_mapping)
    
    rows_with_empty_location = df["location"].isna()
    df["title"] = df["title"].str.lstrip("Mieszkanie Gdańsk").str.lstrip("Nowe mieszkanie").str.strip().str.lower()
    df.loc[rows_with_empty_location, "location"] = df.loc[rows_with_empty_location, "title"].map(gratka_title_to_location)
    
    df.drop(
        labels=["title"], 
        inplace=True,
        axis=1
    )
    
    df.dropna(subset=["location"], inplace=True)

    return df

In [423]:
df_dirty = dirty_data.copy()

In [424]:
df = clear_gratka(df_dirty)

In [427]:
df.head()

,rent,location,surface,rooms,level,building_type
0,6500.0,Srodmiescie - Stare Miasto - Dolne Miasto - Dł...,68.69,3,4,apartment_building
1,2450.0,Brzezno,47.00,2,3,unit
2,6800.0,Srodmiescie - Stare Miasto - Dolne Miasto - Dł...,100.40,4,1,tenement
3,5400.0,Oliwa - Strzyza,125.00,4,2,tenement
5,6800.0,Srodmiescie - Stare Miasto - Dolne Miasto - Dł...,100.40,4,2,tenement
